<a href="https://colab.research.google.com/github/anuragisi/Data_Pipeline_News_Sentiment_Analysis/blob/main/Automated_Data_Pipeline_News_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install newspaper4k lxml-html-clean vaderSentiment fastapi

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 2.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=c91f2c82c9b24f12b0619041e208454fc7a41c8190238fef24edf7def302a6fe
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


# STEP 1: Web Scraping (News Collection)
Use newspaper3k to scrape articles from different news portals.

In [2]:
import newspaper
toi_paper = newspaper.build('https://timesofindia.indiatimes.com/news')

for article in toi_paper.articles:
  print(article.url)

https://timesofindia.indiatimes.com/videos/news
https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news
https://timesofindia.indiatimes.com/entertainment/english/hollywood/news
https://timesofindia.indiatimes.com/web-series/news
https://timesofindia.indiatimes.com/web-series/news/anime
https://timesofindia.indiatimes.com/sports/cricket/england-tour-of-india-2025
https://www.indiatimes.com/worth/news
https://www.indiatimes.com/technology/news
https://www.indiatimes.com/remote-working-work-from-home-culture
https://www.indiatimes.com/trending/king-the-land-actor-lee-jun-ho-dances-with-cane-at-fan-concert-despite-his-ligament-injury-impresses-fans-with-his-dedication-652396.html
https://www.indiatimes.com/trending/just-weeks-before-the-trauma-code-heroes-on-calls-premiere-ju-ji-hoon-was-named-as-one-of-the-worst-actors-of-2024-652395.html
https://www.indiatimes.com/events/valentines-week-2025-full-list-rose-day-propose-day-and-kiss-day-how-to-celebrate-these-days-650824.htm

In [3]:
from newspaper import Article

# Function to extract article text
def get_article_content(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        return f"Error processing {url}: {e}"

# List to store extracted articles
article_texts = []

# Loop through articles and extract content
for article in toi_paper.articles[:50]:
    content = get_article_content(article.url)
    if content:  # Ensure valid content before appending
        article_texts.append(content)

In [5]:
# Print the first 500 characters of each article
for idx, text in enumerate(article_texts):
    print(f"\nArticle {idx + 1}:\n{text[:500]}...\n")


Article 1:
Breaking news about Bollywood celebrities to entertaining gossip and all the updates about film releases - this is your one stop destination to all things movies. From who is making their big debut in Bollywood to which movie is setting the box office on fire and new film announcements - We have got you covered! We also bring to you all the deets about who is shooting where and exclusive paparazzi coverage of Bollywood celebrities. Engaging Bollywood interviews, box office details, interesting t...


Article 2:
Check out all the latest news, gossip and trending news from Hollywood out here. From new posters and trailers to film announcements, this page is a one-stop destination for all stories related to Hollywood movies and celebrities....


Article 3:
Ranveer Allahbadia and Nikki Sharma spark breakup rumors as the alleged couple unfollow each other on social media

Mohnish Bahl terms working with Sooraj Barjatya 'nothing short of a blessing

'Oops! Ab Kya'? trailer gives 

In [9]:
import psycopg2

def save_to_db(article_number, article_text, sentiment):
    try:
        conn = psycopg2.connect("dbname=news user=postgres password=yourpassword")
        cur = conn.cursor()

        cur.execute("""
            INSERT INTO articles (article_number, content, sentiment)
            VALUES (%s, %s, %s)
        """, (article_number, article_text, sentiment))

        conn.commit()
        cur.close()
        conn.close()

        print(f"Article {article_number} saved successfully!")

    except Exception as e:
        print(f"Error saving article {article_number}: {e}")

In [8]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment
def analyze_sentiment(text):
    score = analyzer.polarity_scores(text)

    if score['compound'] >= 0.05:
        return "Positive"
    elif score['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

In [10]:
# Loop through extracted articles and print sentiment
for idx, text in enumerate(article_texts):
    sentiment = analyze_sentiment(text)
    save_to_db(idx + 1, text, sentiment)
    print(f"Article {idx + 1} Sentiment: {sentiment}")

Error saving article 1: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

Article 1 Sentiment: Positive
Error saving article 2: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

Article 2 Sentiment: Negative
Error saving article 3: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

Article 3 Sentiment: Positive
Error saving article 4: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

Article 4 Sentiment: Positive
Error saving article 5: connection to server on socket "/var/run/postgresql/.s.PGSQL

In [19]:
from fastapi import FastAPI
import psycopg2

app = FastAPI()

def get_sentiment_summary():
    conn = psycopg2.connect("dbname=newsdb user=postgres password=yourpassword")
    cur = conn.cursor()
    cur.execute("SELECT sentiment, COUNT(*) FROM news_articles GROUP BY sentiment")
    result = cur.fetchall()
    conn.close()
    return dict(result)

@app.get("/sentiment-summary")
def sentiment_summary():
    return get_sentiment_summary()